<img src="files/Pics/LOGOS.png" width="800">

# Labor 02: Többváltozós lineáris regresszió

A gyakorlat során a L01-es laboron bevezetett egyváltozós modellónket fogjuk bővíteni több változós esetre.

### Ingatlanárak:

Ebben a feladatban ingatlanárak becslésére fogunk többváltozós lineáris regressziót implementálni.

Tegyük fel, hogy egy lakást szeretnénk eladni, ám ehhez szeretnénk tudni a lakás valós értékét, hogy ne veszítsünk az eladáskor. Egy lehetséges módja ennek, hogy adatot gyűjtve, majd az adatok alapján modellt készítve, becsüljük meg a lakás ingatlanpiaci árát. Adataink az ingatlan területe ($m^2$) és a szobák száma (db), illetve az értékesítéskor meghatározott ár ($) lesz.

### 1: Importjáljuk be a számunkra fontos csomagokat!
Szükségünk lesz:
- NumPy -ra a tömbkezeléshez
- MatPlotLib pyplot csomagjára a megjelenítéshez
- Pandas -ra az adatbeolvasáshoz

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Töltsük be az adatainkat! Használjuk ehhez a Pandas csomagot majd alakítsuk numpy tömbbé.

In [ ]:
data = pd.read_csv('Lab2data.txt',header = None).to_numpy()    # adatok beolvasása és NumPy tömbbé alakítása
X = data[:,0:2]                                                # X rendezése
m = X.shape[0]                                                 # adatok száma
Y = data[:,2].reshape(m,1)                                     # Y rendezése oszlopvektorba

print('X:',X.shape)                                            # adattömbök méretének / adatok számának kiírása
print('Y:',Y.shape)
print('Adatok száma',m)

### 2: Normalizáljuk az értékeinket (feature scaling & mean normalization)

Az adatainkról elképzelhető, hogy más nagyságrendekben találhatóak. Esetünkben az ingatlan területe és a szobák száma között könnyen beláthatjuk, hogy legalább egy nagyságrendi különbség van. Ilyenkor érdemes az értékeinket normalizálni, hogy egy nagyságrendbe essenek és minden bemeneti változó a [-1..1] vagy [0..1] intervallumban mozogjon. Ez a művelet a konvergenciát fogja elősegíteni, mivel nem lesz dominánsan jelen lévő változó, ami elnyomhatná a többi változó hatását. <br>
Ehhez mi az alábbi összefüggést fogjuk használni (Feature scaling and Mean normalization):

$ x = \frac{x - mean(x)}{std(x)} $

, vagyis egy adott mintából kivonjuk a minták átlagát (mean normalization) és leosztjuk a minták szórásával (feature scaling).

Grafikusan elképzelve az origó körül könnyebben megtaláljuk azt az egyenest, ami a hipotézisünknek megfelelően lefedi az adatainkat. Célszerű tehát az adatainkat ebbe a régióba trasformálni.

<img src="files/Pics/L02_Scaling.png" width="250">

Hozzuk létre a normalizáló függvényt!

In [ ]:
def featureNormalize(X):
######################################################    
    
    
    
    
######################################################
    return X_norm, avg, sigma                            # képlet alapján eredmény visszaadása

print('Normalizing X vector ...')                       
X_norm,avg,sigma = featureNormalize(X)                  # Normalizálás
X_norm=np.column_stack((np.ones(m),X_norm))             # bias hozzáadása

Normalizálás után a bemeneti X mátrixhoz a BIAS tagokat is hozzáfűzzük.

### 3: Gradiens módszer
Az előző gyakorlat mintájára létrehozzuk a gradiens módszert több változós formába! Adat struktúránk a következő képpen alakul.
<img src="files/Pics/L02_Matrixok.png" width="500">

Hipotézis függvényünk több változós esetre a következő képpen írható fel:

$ h_{w}(x)=w_0x_0+w_1x_1+w_2x_2+ ... +w_nx_n $ <br>

Mátrix műveletekkel pedig:

<img src="files/Pics/L02_XW.png" width="550">

A költség függvény képlete: <br>

$ C(W)=C(w_0,w_1,...,w_n)=\frac{1}{2m}\sum_{i=1}^{m}(h_w(x^i)-y^i)^2 $

Tipp: Programozás során érdemes kihasználni

<img src="files/Pics/L02_Sum.png" width="550">

A gradiens módszer általános súlyfrissítési képlete:

$ \color{red}{(j=0...n)}\hspace{7mm} w_j:=w_j-\mu\frac{1}{2m}\sum_{i=1}^{m}(h_w(x^i)-y^i)\cdot x_j^i $

$\color{red}{A\ szimultán\ frissítésre\ figyelni\ kell!}$

In [ ]:
# Költségfüggvény
def computeCostMulti(X,Y,W):
#############################################     
   
    
############################################# 
    return C

In [ ]:
# Grádiens módszer
def gradientDescentMulti(X,Y,W,lr,epochs):              
#############################################    
    
    
    
    
    
    
    
    
############################################# 
    return W, C_history

In [ ]:
print('Running gradient descent ...')
lr = 0.015                                                  # tanulási ráta
epochs = 1200                                               # epoch szám
W=np.zeros((3,1))                                           # kezdeti súly (0;0;0)
W,C_history= gradientDescentMulti(X_norm,Y,W,lr,epochs)     # Grádiens módszer használata
print('''Weights expected from gradient descent (approx.):
 [[340412.65505089]
 [110607.33011833]
 [ -6625.8311691 ]]
''')
print('Weights computed from gradient descent:\n', W)

if int(W[0]) == 340412 and int(W[1]) == 110607 and int(W[2]) == -6625:
    print("\n A gradientDescentMulti() függvény megfelelő. Tovább mehet.")
else:
    print("\n Valami nem stimmel. Korrekció szükséges!")

Ellenőrizzük a konvergenciát egy ábra segítségével!

In [ ]:
plt.plot(C_history)                                                                 # C_history kirajzolása
plt.title("Gradient descent algorithms effect through the iterations",pad= 20)
plt.xlabel("Iterations")
plt.ylabel("Cost function value")
plt.show()

### 4: Becslés
Becsüljük meg egy 1650 $m^2$-es, 3 szobás ingatlan árát! Figyeljünk, hogy az adatok normalizálását itt is végezzük el.

In [ ]:
def predict(FEET, BED):
#############################################    
    
############################################# 
    return price

In [ ]:
FEET = 1650
BED = 3
price = predict(FEET, BED)
print('''Prediction for a 1650 sq-ft / 3 bedroom house:
(predicted price should be approx. $29300) %.2f''' % price)

## Kicsit másként. Magasabb szintű csomagokkal

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

data = pd.read_csv('Lab2data.txt',header = None)                        # adatok beolvasása
X = data.iloc[:, 0:2].values.reshape(-1,2)                              # X rendezése
Y = data.iloc[:, 2].values.reshape(-1,1)                                # Y rendezése

lin_reg = LinearRegression()                                            # lineáris regressziós modell class létrehozása
lin_reg.fit(X,Y)                                                        # illesztés az X,Y alapján

pred = lin_reg.predict([[1650,3]])                                      # predikció 1650 nm és 3 szobás lakásra
print('Prediction for a 1650 sq-ft / 3 bedroom house:\n %.2f' % pred)